##### Libraries

In [1]:
import numpy as np
import pandas as pd
import sys
from pathlib import Path
# Get current working directory
current_dir = Path.cwd()
sys.path.append(str(current_dir.parent/"data"))
print("Current working directory:", current_dir)

Current working directory: c:\Users\TEMP\OneDrive\PPGAO_2024_2027\2025\UAS-Planner\Godot-UAV-Simulator\sandbox


##### Process CSV data and extract Voxels parameters

In [2]:
class DataProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.pd_df = None
        self.np_array = None

    def process_csv(self):
        self.pd_df = pd.read_csv(self.file_path)
        self.np_array = self.pd_df.to_numpy()
        if self.np_array is not None:
            latitudes = self.np_array[:, 1]
            longitudes = self.np_array[:, 2]
            altitudes = self.np_array[:, 0]
            max_lat = latitudes.max()
            min_lat = latitudes.min()
            max_lon = longitudes.max()
            min_lon = longitudes.min()
            max_alt = altitudes.max()
            min_alt = altitudes.min()
        else:
            raise ValueError("Data not loaded. Please read the CSV file first.")
        return (min_lat, max_lat, min_lon, max_lon, min_alt, max_alt)
        

In [3]:
csv_file_path = "C:\\Users\\TEMP\\OneDrive\\PPGAO_2024_2027\\2025\\UAS-Planner\\Godot-UAV-Simulator\\data\\Filtered_FAA_UAS_FacilityMap_Data_LGA.csv"
csv_to_nparray = DataProcessor(csv_file_path)
csv_to_nparray.process_csv()

min_lat,max_lat, min_lon, max_lon, min_alt, max_alt = csv_to_nparray.process_csv()

print(type(csv_to_nparray.np_array))
print(csv_to_nparray.np_array)
print(f"Min Latitude: {min_lat}, Max Latitude: {max_lat}")
print(f"Min Longitude: {min_lon}, Max Longitude: {max_lon}")
print(f"Min Altitude: {min_alt}, Max Altitude: {max_alt}")
print("CSV file processed successfully.")

<class 'numpy.ndarray'>
[[400.          40.55417343 -73.91250593]
 [400.          40.55417343 -73.90417259]
 [400.          40.55417343 -73.89583926]
 ...
 [400.          40.88750683 -73.87083925]
 [400.          40.88750683 -73.86250592]
 [400.          40.88750683 -73.85417258]]
Min Latitude: 40.55417343, Max Latitude: 40.88750683
Min Longitude: -73.99583928, Max Longitude: -73.5958392
Min Altitude: 0.0, Max Altitude: 400.0
CSV file processed successfully.


In [4]:
class VoxelDataFrame:
    
    def __init__(self,input_np_array, voxel_size, voxel_edge_size, min_lat, min_lon, min_alt):
        """Initialize the VoxelDataFrame with a numpy array."""
        self.input_np_array = input_np_array
        self.id = np.zeros(voxel_size, dtype=tuple)
        self.position_meters = np.zeros(voxel_size, dtype=tuple)
        self.position_degrees = np.zeros(voxel_size, dtype=tuple)
        self.available: bool = None
        self.forbiden: bool = None
        self.voxel_size = voxel_size
        self.voxel_edge_size = voxel_edge_size
        self.min_lat: float = min_lat
        self.min_lon: float = min_lon
        self.min_alt: float = min_alt
    
    def build_voxel_dataframe(self):
        """Build the voxel dataframe from the input numpy array."""
        
        if self.input_np_array is not None:
            shape = self.input_np_array.shape

            # Create grid indices for all voxels
            i, j, k = np.indices(self.voxel_size)

            # Assign voxel ids
            self.id = np.stack((i, j, k), axis=-1)

            # Calculate positions in meters (center of each voxel)
            self.position_meters = np.stack(
                (i * self.voxel_edge_size / 2, j * self.voxel_edge_size / 2, k * self.voxel_edge_size / 2),
                axis=-1
            )

            # Calculate positions in degrees
            latitudes = self.min_lat-0.008 + ((i * self.voxel_edge_size / 111320)/2)
            longitudes = self.min_lon-0.008 + ((j * self.voxel_edge_size / (111320 * np.cos(np.radians(self.min_lat))))/2)
            altitudes = self.min_alt + (k * self.voxel_edge_size/2 )
            self.position_degrees = np.stack((latitudes, longitudes, altitudes), axis=-1)


In [5]:
#Voxels dimensions calculations
voxel_edge_size = 60  # Edge size of the voxel in meters

# Calculate the number of voxels in each dimension
num_voxels_lat = int((max_lat - min_lat+0.016)*111320.0 / voxel_edge_size)
num_voxels_lon = int((max_lon - min_lon+0.016)*111320.0*np.cos(np.radians((max_lat + min_lat) / 2)) / voxel_edge_size)
num_voxels_alt = int((max_alt - min_alt)*0.3048 / voxel_edge_size)

voxel_np_array_size = (num_voxels_lat, num_voxels_lon, num_voxels_alt)


print(f"Number of voxels in latitude: {num_voxels_lat}")
print(f"Number of voxels in longitude: {num_voxels_lon}")
print(f"Number of voxels in altitude: {num_voxels_alt}")
print(f"Voxel array dimensions : {voxel_np_array_size}")

Number of voxels in latitude: 648
Number of voxels in longitude: 584
Number of voxels in altitude: 2
Voxel array dimensions : (648, 584, 2)


In [6]:
voxel_df = VoxelDataFrame(csv_file_path, min_lat, max_lat, min_lon, max_lon)
voxel_df.read_csv(csv_file_path)
voxel_df.print_numpy_data()

TypeError: VoxelDataFrame.__init__() missing 1 required positional argument: 'min_alt'